In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from datetime import datetime
import pytz
from pytz import timezone
from datetime import datetime, date, timedelta
import sqlalchemy

# from config import open_weather_key

In [2]:
## Cities dataframe (through webscraping)
def recreate_wiki(cities):
    # Initialize an empty list that will be filled with one dictionary of information per city
    city_data = []

    # Iterate through the list of cities to collect information
    for city in cities:
        # Construct the Wikipedia URL for the city
        url = f"https://en.wikipedia.org/wiki/{city}"

        # Send a GET request and parse the HTML with BeautifulSoup
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")

        # Initialize an empty dictionary for the city's information
        city_info = {}

        # Extract relevant information from the HTML using selectors
        city_info["city"] = soup.select(".firstHeading")[0].get_text()
        city_info["country"] = soup.select(".infobox-data")[0].get_text()
        city_info["latitude"] = soup.select(".latitude")[0].get_text()
        city_info["longitude"] = soup.select(".longitude")[0].get_text()
        city_info["website"] = (
            soup.select_one('.infobox-label:-soup-contains("Website")')
            .find_next(class_="infobox-data")
            .get_text()
        )

        # Check if elevation information is available on the page
        elevation_info = soup.select_one('.infobox-label:-soup-contains("Elevation")')
        if elevation_info:
            city_info["elevation"] = elevation_info.find_next(
                class_="infobox-data"
            ).get_text()

        # Check if population information is available on the page
        population_info = soup.select_one(
            'th.infobox-header:-soup-contains("Population")'
        )
        if population_info:
            city_info["population"] = population_info.parent.find_next_sibling().find(
                string=re.compile(r"\d+")
            )

        # Add the city's dictionary to the list
        city_data.append(city_info)

    # Create a DataFrame from the list of dictionaries
    cities_df = pd.DataFrame(city_data)

    # Fix formatting issues in latitude and longitude columns
    cities_df["latitude"] = (
        cities_df["latitude"]
        .str.replace("°", ".")
        .str.replace("′", "")
        .str.replace("″", "")
    )
    cities_df["longitude"] = (
        cities_df["longitude"]
        .str.replace("°", ".")
        .str.replace("′", "")
        .str.replace("″", "")
    )

    # Return the DataFrame
    return cities_df

In [3]:
## Cities dataframe (through webscraping)
def recreate_wiki(cities):
    # Initialize an empty list that will be filled with one dictionary of information per city
    city_data = []

    # Iterate through the list of cities to collect information
    for city in cities:
        # Construct the Wikipedia URL for the city
        url = f"https://en.wikipedia.org/wiki/{city}"

        # Send a GET request and parse the HTML with BeautifulSoup
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")

        # Initialize an empty dictionary for the city's information
        city_info = {}

        # Extract relevant information from the HTML using selectors
        city_info["city"] = soup.select(".firstHeading")[0].get_text()
        city_info["country"] = soup.select(".infobox-data")[0].get_text()
        city_info["latitude"] = soup.select(".latitude")[0].get_text()
        city_info["longitude"] = soup.select(".longitude")[0].get_text()
        city_info["website"] = (
            soup.select_one('.infobox-label:-soup-contains("Website")')
            .find_next(class_="infobox-data")
            .get_text()
        )

        # Check if elevation information is available on the page
        elevation_info = soup.select_one('.infobox-label:-soup-contains("Elevation")')
        if elevation_info:
            city_info["elevation"] = elevation_info.find_next(
                class_="infobox-data"
            ).get_text()

        # Check if population information is available on the page
        population_info = soup.select_one(
            'th.infobox-header:-soup-contains("Population")'
        )
        if population_info:
            city_info["population"] = population_info.parent.find_next_sibling().find(
                string=re.compile(r"\d+")
            )

        # Add the city's dictionary to the list
        city_data.append(city_info)

    # Create a DataFrame from the list of dictionaries
    cities_df = pd.DataFrame(city_data)

    # Fix formatting issues in latitude and longitude columns
    cities_df["latitude"] = (
        cities_df["latitude"]
        .str.split("″")
        .str[0]
        .str.replace("°", ".", regex=False)
        .str.replace("′", "", regex=False)
    )

    cities_df["longitude"] = (
        cities_df["longitude"]
        .str.split("″")
        .str[0]
        .str.replace("°", ".", regex=False)
        .str.replace("′", "", regex=False)
    )

    # Return the DataFrame
    return cities_df

In [4]:
list_of_cities = ["Berlin", "Amsterdam", "London", "Manchester", "Barcelona"]
cities_final = recreate_wiki(list_of_cities)
cities_final

,city,country,latitude,longitude,website,elevation,population
0,Berlin,Germany,52.3112,13.2418,berlin.de,34 m (112 ft),"3,850,809"
1,Amsterdam,Netherlands,52.2222,04.5337,www.amsterdam.nl,−2 m (−7 ft),"921,402"
2,London,United Kingdom,51.3026,0.739,www.london.gov.uk,36 ft (11 m),"8,799,800"
3,Manchester,United Kingdom,53.2846,2.1443,manchester.gov.uk,125 ft (38 m),"549,853"
4,Barcelona,Spain,41.2258,02.1037,www.barcelona.cat,12 m (39 ft),"1,620,343"


In [29]:
cities_final["latitude"]
cities_final["longitude"]

0    13.2418E
1      10.00E
2      0.739W
3     2.1443W
4    02.1037E
Name: longitude, dtype: object

In [5]:
schema = "gans_wbs"
host = "host"
user = "admin"
password = "password"
port = 3306
con = f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}"

In [7]:
cities_final.to_sql("cities_final", if_exists="append", con=con, index=False)

5

In [8]:
# using this to extract city id
cities_df = pd.read_sql_table("cities_final", con=con)
cities_df.head()

,city_id,city,country,latitude,longitude,website,elevation,population
0,1,Berlin,Germany,52.3112,13.2418,berlin.de,34 m (112 ft),"3,850,809"
1,2,Amsterdam,Netherlands,52.2222,04.5337,www.amsterdam.nl,−2 m (−7 ft),"921,402"
2,3,London,United Kingdom,51.3026,0.739,www.london.gov.uk,36 ft (11 m),"8,799,800"
3,4,Manchester,United Kingdom,53.2846,2.1443,manchester.gov.uk,125 ft (38 m),"549,853"
4,5,Barcelona,Spain,41.2258,02.1037,www.barcelona.cat,12 m (39 ft),"1,620,343"


In [9]:
## Weather dataframe
def get_weather_loop(cities):
    API_key = "key"

    tz = pytz.timezone("Europe/Berlin")
    now = datetime.now().astimezone(tz)

    weather_dict = {
        "city": [],
        "country": [],
        "forecast_time": [],
        "outlook": [],
        "detailed_outlook": [],
        "temperature": [],
        "temperature_feels_like": [],
        "clouds": [],
        "rain": [],
        "snow": [],
        "wind_speed": [],
        "wind_deg": [],
        "humidity": [],
        "pressure": [],
        "information_retrieved_at": [],
    }

    for city in cities:
        url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric"
        response = requests.get(url)
        json = response.json()

        for i in json["list"]:
            weather_dict["city"].append(json["city"]["name"])
            weather_dict["country"].append(json["city"]["country"])
            weather_dict["forecast_time"].append(i["dt_txt"])
            weather_dict["outlook"].append(i["weather"][0]["main"])
            weather_dict["detailed_outlook"].append(i["weather"][0]["description"])
            weather_dict["temperature"].append(i["main"]["temp"])
            weather_dict["temperature_feels_like"].append(i["main"]["feels_like"])
            weather_dict["clouds"].append(i["clouds"]["all"])
            try:
                weather_dict["rain"].append(i["rain"]["3h"])
            except:
                weather_dict["rain"].append("0")
            try:
                weather_dict["snow"].append(i["snow"]["3h"])
            except:
                weather_dict["snow"].append("0")
            weather_dict["wind_speed"].append(i["wind"]["speed"])
            weather_dict["wind_deg"].append(i["wind"]["deg"])
            weather_dict["humidity"].append(i["main"]["humidity"])
            weather_dict["pressure"].append(i["main"]["pressure"])
            weather_dict["information_retrieved_at"].append(
                now.strftime("%d/%m/%Y %H:%M:%S")
            )

    return pd.DataFrame(weather_dict)

In [10]:
# need to do something with weather key document
get_weather_loop(list_of_cities)
weather_1 = get_weather_loop(list_of_cities)

weather_1

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,2023-11-08 12:00:00,Clouds,broken clouds,10.19,9.30,83,0,0,5.60,255,78,1004,08/11/2023 10:53:48
1,Berlin,DE,2023-11-08 15:00:00,Clouds,overcast clouds,10.29,9.23,92,0,0,4.59,244,71,1008,08/11/2023 10:53:48
2,Berlin,DE,2023-11-08 18:00:00,Clouds,overcast clouds,8.60,6.09,87,0,0,4.40,216,73,1013,08/11/2023 10:53:48
3,Berlin,DE,2023-11-08 21:00:00,Clouds,scattered clouds,7.52,4.61,50,0,0,4.71,208,72,1012,08/11/2023 10:53:48
4,Berlin,DE,2023-11-09 00:00:00,Clouds,broken clouds,7.70,4.87,75,0,0,4.65,199,70,1010,08/11/2023 10:53:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Barcelona,ES,2023-11-12 21:00:00,Clouds,overcast clouds,17.41,17.01,100,0,0,1.12,112,69,1014,08/11/2023 10:53:48
196,Barcelona,ES,2023-11-13 00:00:00,Clouds,overcast clouds,17.59,17.10,100,0,0,3.38,298,65,1014,08/11/2023 10:53:48
197,Barcelona,ES,2023-11-13 03:00:00,Clouds,overcast clouds,16.95,16.42,95,0,0,3.43,316,66,1013,08/11/2023 10:53:48
198,Barcelona,ES,2023-11-13 06:00:00,Clouds,overcast clouds,15.94,15.42,95,0,0,2.07,41,70,1016,08/11/2023 10:53:48


In [11]:
weather_1 = weather_1.merge(
    cities_df[["city_id", "city"]], how="left"
)  # merging cities_df for city_id
# keeping only the columns that we have in weather table in sql
weather_2 = weather_1[
    [
        "city_id",
        "city",
        "country",
        "forecast_time",
        "outlook",
        "temperature",
        "temperature_feels_like",
        "wind_speed",
        "rain",
    ]
]
# checking if datatype forecast is datetime
weather_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_id                 200 non-null    int64  
 1   city                    200 non-null    object 
 2   country                 200 non-null    object 
 3   forecast_time           200 non-null    object 
 4   outlook                 200 non-null    object 
 5   temperature             200 non-null    float64
 6   temperature_feels_like  200 non-null    float64
 7   wind_speed              200 non-null    float64
 8   rain                    200 non-null    object 
dtypes: float64(3), int64(1), object(5)
memory usage: 14.2+ KB


In [12]:
weather_2["forecast_time"] = pd.to_datetime(weather_2["forecast_time"])
weather_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   city_id                 200 non-null    int64         
 1   city                    200 non-null    object        
 2   country                 200 non-null    object        
 3   forecast_time           200 non-null    datetime64[ns]
 4   outlook                 200 non-null    object        
 5   temperature             200 non-null    float64       
 6   temperature_feels_like  200 non-null    float64       
 7   wind_speed              200 non-null    float64       
 8   rain                    200 non-null    object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 14.2+ KB


C:\Users\k.ritscher\AppData\Local\Temp\ipykernel_6952\3286763254.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_2["forecast_time"] = pd.to_datetime(weather_2["forecast_time"])


In [13]:
weather_2.to_sql("weather_2", if_exists="append", con=con, index=False)

200

In [14]:
weather_2 = pd.read_sql_table("weather_2", con=con)
weather_2.sample(5)

,city_id,city,country,forecast_time,outlook,temperature,temperature_feels_like,wind_speed,rain
60,2,Amsterdam,NL,2023-11-11 00:00:00,Rain,8.53,5.83,4.82,1.54
164,5,Barcelona,ES,2023-11-09 00:00:00,Clouds,14.82,13.95,4.31,0
98,3,London,GB,2023-11-10 18:00:00,Clouds,7.08,3.88,5.14,0
71,2,Amsterdam,NL,2023-11-12 09:00:00,Clouds,6.19,4.58,2.19,0
19,1,Berlin,DE,2023-11-10 21:00:00,Clouds,6.88,5.14,2.49,0


## Checkpoint: Population of cities and weather tables are working

### Airport Codes

In [28]:
def icao_airport_codes(latitudes, longitudes):
    # assert len(latitudes) == len(longitudes)

    list_for_df = []

    for index, value in enumerate(latitudes):
        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{value}/{longitudes[index]}/km/100/16"

        querystring = {"withFlightInfoOnly": "true"}

        headers = {
            "X-RapidAPI-Host": "host",
            "X-RapidAPI-Key": "key",
        }

        response = requests.request("GET", url, headers=headers, params=querystring)

        list_for_df.append(pd.json_normalize(response.json()["items"]))

    return pd.concat(list_for_df, ignore_index=True)


latitudes = [52.5200]  # here can also be a list of coordinates
longitudes = [13.4050]  # here can also be a list of coordinates
icao_airport_codes(latitudes, longitudes)

,icao,iata,name,shortName,municipalityName,countryCode,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,52.35139,13.493889


In [31]:
airport_codes = icao_airport_codes(latitudes, longitudes)

airport_codes.rename(
    columns={"icao": "airport_icao", "name": "airport_name"}, inplace=True
)
airport_codes = airport_codes[
    [
        "airport_icao",
        "airport_name",
    ]
]
airport_codes

,airport_icao,airport_name
0,EDDB,Berlin Brandenburg


In [47]:
# icao_list = airport_codes["airport_icao"].tolist()

['EDDB']

In [32]:
airport_codes.to_sql("airport_codes", if_exists="append", con=con, index=False)

1

## Checkpoint: Airport Codes

In [17]:
# flights
def tomorrows_flight_arrivals(icao_list):
    # Get today's date in Berlin timezone
    today = datetime.now().astimezone(timezone("Europe/Berlin")).date()
    # Calculate tomorrow's date
    tomorrow = today + timedelta(days=1)

    # Initialize an empty list to store flight data
    list_for_df = []

    # Loop over each ICAO code in the input list
    for icao in icao_list:
        # Define the two time periods for which to fetch data
        times = [["00:00", "11:59"], ["12:00", "23:59"]]

        # Loop over each time period
        for time in times:
            # Construct the URL for the API request
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            # Define the query parameters for the API request
            querystring = {
                "withLeg": "true",
                "direction": "Arrival",
                "withCancelled": "false",
                "withCodeshared": "true",
                "withCargo": "false",
                "withPrivate": "false",
            }
            # Define the headers for the API request
            headers = {
                "x-rapidapi-host": "host",
                "x-rapidapi-key": "key",
            }
            # Make the API request
            response = requests.request("GET", url, headers=headers, params=querystring)
            # Parse the JSON response
            flights_json = response.json()

            # Loop over each flight in the response
            for flight in flights_json["arrivals"]:
                # Initialize an empty dictionary to store flight data
                flights_dict = {}
                # Store the ICAO code and flight data in the dictionary
                flights_dict["arrival_icao"] = icao
                # Use the .get() method to avoid KeyError if a key doesn't exist in the dictionary
                flights_dict["arrival_time_local"] = flight["arrival"].get(
                    "scheduledTimeLocal", None
                )
                flights_dict["arrival_terminal"] = flight["arrival"].get(
                    "terminal", None
                )
                flights_dict["departure_city"] = flight["departure"]["airport"].get(
                    "name", None
                )
                flights_dict["departure_icao"] = flight["departure"]["airport"].get(
                    "icao", None
                )
                flights_dict["departure_time_local"] = flight["departure"].get(
                    "scheduledTimeLocal", None
                )
                flights_dict["airline"] = flight["airline"].get("name", None)
                flights_dict["flight_number"] = flight.get("number", None)
                # Store the current date in Berlin timezone
                flights_dict["data_retrieved_on"] = (
                    datetime.now().astimezone(timezone("Europe/Berlin")).date()
                )
                # Append the flight dictionary to the list
                list_for_df.append(flights_dict)

    # Convert the list of flight dictionaries to a DataFrame and return it
    return pd.DataFrame(list_for_df)

In [48]:
icao_list = airport_codes["airport_icao"].tolist()
flights = tomorrows_flight_arrivals(icao_list)
flights = flights[
    [
        "flight_number",
        "departure_icao",
        "arrival_icao",
        "arrival_time_local",
        "departure_city",
    ]
]
flights

,flight_number,departure_icao,arrival_icao,arrival_time_local,departure_city
0,E4 546,LIRF,EDDB,None,Rome
1,QR 79,OTHH,EDDB,None,Doha
2,SM 2962,HEMA,EDDB,None,Marsa Alam
3,SR 8853,LIRF,EDDB,None,Rome
4,BT 211,EVRA,EDDB,None,Riga
...,...,...,...,...,...
254,EC 5364,HEMA,EDDB,None,Marsa Alam
255,EW 8983,HEGN,EDDB,None,Hurghada
256,I2 3674,LEMD,EDDB,None,Madrid
257,IB 3674,LEMD,EDDB,None,Madrid


In [49]:
flights.to_sql("flights", if_exists="append", con=con, index=False)

259

In [51]:
flights = pd.read_sql_table("flights", con=con)
flights

,flight_number,departure_icao,arrival_icao,arrival_time_local,departure_city
0,E4 546,LIRF,EDDB,None,Rome
1,QR 79,OTHH,EDDB,None,Doha
2,SM 2962,HEMA,EDDB,None,Marsa Alam
3,SR 8853,LIRF,EDDB,None,Rome
4,BT 211,EVRA,EDDB,None,Riga
...,...,...,...,...,...
254,EC 5364,HEMA,EDDB,None,Marsa Alam
255,EW 8983,HEGN,EDDB,None,Hurghada
256,I2 3674,LEMD,EDDB,None,Madrid
257,IB 3674,LEMD,EDDB,None,Madrid


## Checkpoint flights

In [54]:
get_arrivals(icao_list)

,arrival_icao,flight_number,departure_name,departure_icao,arrival_time
0,EDDB,AY 1433,Helsinki,EFHK,2023-11-04 12:20+01:00
1,EDDB,EC 5108,Barcelona,LEBL,2023-11-04 12:10+01:00
2,EDDB,EC 5116,Palma De Mallorca,LEPA,2023-11-04 12:15+01:00
3,EDDB,EC 5120,Palma De Mallorca,LEPA,2023-11-04 12:15+01:00
4,EDDB,FR 2419,Tenerife Island,GCTS,2023-11-04 12:10+01:00
...,...,...,...,...,...
1868,LEBL,VY 7825,London,EGKK,2023-11-04 22:35+01:00
1869,LEBL,VY 8247,Paris,LFPG,2023-11-04 22:45+01:00
1870,LEBL,VY 3719,Menorca Island,LEMH,2023-11-04 22:20+01:00
1871,LEBL,VY 3919,Palma De Mallorca,LEPA,2023-11-04 22:20+01:00


In [1]:
def icao_airport_codes(latitudes, longitudes):
    airports_list = []

    for latitudes, longitudes in zip(latitudes, longitudes):
        url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

        querystring = {
            "lat": f"{latitudes}",
            "lon": f"{longitudes}",
            "radiusKm": "50",
            "limit": "7",
            "withFlightInfoOnly": "true",
        }

        headers = {
            "X-RapidAPI-Key": "key",
            "X-RapidAPI-Host": "host",
        }

        response = requests.get(url, headers=headers, params=querystring)

        airports_list.append(pd.json_normalize(response.json()["items"]))
    return pd.concat(airports_list, ignore_index=True)

# Checkpoint Creating the Lambda Function

In [15]:
def get_weather_loop_2(cities):
    API_key = "key"

    tz = pytz.timezone("Europe/Berlin")
    now = datetime.now().astimezone(tz)

    weather_dict = {
        "city": [],
        "country": [],
        "forecast_time": [],
        "outlook": [],
        "detailed_outlook": [],
        "temperature": [],
        "temperature_feels_like": [],
        "clouds": [],
        "rain": [],
        "snow": [],
        "wind_speed": [],
        "wind_deg": [],
        "humidity": [],
        "pressure": [],
        "information_retrieved_at": [],
    }

    for city in cities:
        url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric"
        response = requests.get(url)
        json = response.json()

        for i in json["list"]:
            weather_dict["city"].append(json["city"]["name"])
            weather_dict["country"].append(json["city"]["country"])
            weather_dict["forecast_time"].append(i["dt_txt"])
            weather_dict["outlook"].append(i["weather"][0]["main"])
            weather_dict["detailed_outlook"].append(i["weather"][0]["description"])
            weather_dict["temperature"].append(i["main"]["temp"])
            weather_dict["temperature_feels_like"].append(i["main"]["feels_like"])
            weather_dict["clouds"].append(i["clouds"]["all"])
            try:
                weather_dict["rain"].append(i["rain"]["3h"])
            except:
                weather_dict["rain"].append("0")
            try:
                weather_dict["snow"].append(i["snow"]["3h"])
            except:
                weather_dict["snow"].append("0")
            weather_dict["wind_speed"].append(i["wind"]["speed"])
            weather_dict["wind_deg"].append(i["wind"]["deg"])
            weather_dict["humidity"].append(i["main"]["humidity"])
            weather_dict["pressure"].append(i["main"]["pressure"])
            weather_dict["information_retrieved_at"].append(
                now.strftime("%d/%m/%Y %H:%M:%S")
            )

    weather_df = pd.DataFrame(weather_dict)
    cities_df = pd.read_sql_table("cities_final", con=con)
    weather_df = weather_df.merge(cities_df[["city_id", "city"]], how="left")
    weather_df = weather_df[
        [
            "city_id",
            "city",
            "country",
            "forecast_time",
            "outlook",
            "temperature",
            "temperature_feels_like",
            "wind_speed",
            "rain",
        ]
    ]
    weather_df["forecast_time"] = pd.to_datetime(weather_df["forecast_time"])
    weather_df.to_sql("weather_2", if_exists="append", con=con, index=False)

In [ ]:
def tomorrows_flight_arrivals(icao_list):
    # Get today's date in Berlin timezone
    today = datetime.now().astimezone(timezone("Europe/Berlin")).date()
    # Calculate tomorrow's date
    tomorrow = today + timedelta(days=1)

    # Initialize an empty list to store flight data
    list_for_df = []

    # Loop over each ICAO code in the input list
    for icao in icao_list:
        # Define the two time periods for which to fetch data
        times = [["00:00", "11:59"], ["12:00", "23:59"]]

        # Loop over each time period
        for time in times:
            # Construct the URL for the API request
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            # Define the query parameters for the API request
            querystring = {
                "withLeg": "true",
                "direction": "Arrival",
                "withCancelled": "false",
                "withCodeshared": "true",
                "withCargo": "false",
                "withPrivate": "false",
            }
            # Define the headers for the API request
            headers = {
                "x-rapidapi-host": "host",
                "x-rapidapi-key": "key",
            }
            # Make the API request
            response = requests.request("GET", url, headers=headers, params=querystring)
            # Parse the JSON response
            flights_json = response.json()

            # Loop over each flight in the response
            for flight in flights_json["arrivals"]:
                # Initialize an empty dictionary to store flight data
                flights_dict = {}
                # Store the ICAO code and flight data in the dictionary
                flights_dict["arrival_icao"] = icao
                # Use the .get() method to avoid KeyError if a key doesn't exist in the dictionary
                flights_dict["arrival_time_local"] = flight["arrival"].get(
                    "scheduledTimeLocal", None
                )
                flights_dict["arrival_terminal"] = flight["arrival"].get(
                    "terminal", None
                )
                flights_dict["departure_city"] = flight["departure"]["airport"].get(
                    "name", None
                )
                flights_dict["departure_icao"] = flight["departure"]["airport"].get(
                    "icao", None
                )
                flights_dict["departure_time_local"] = flight["departure"].get(
                    "scheduledTimeLocal", None
                )
                flights_dict["airline"] = flight["airline"].get("name", None)
                flights_dict["flight_number"] = flight.get("number", None)
                # Store the current date in Berlin timezone
                flights_dict["data_retrieved_on"] = (
                    datetime.now().astimezone(timezone("Europe/Berlin")).date()
                )
                # Append the flight dictionary to the list
                list_for_df.append(flights_dict)

    flights_df = pd.DataFrame(list_for_df)
    airport_codes = pd.read_sql_table("airport_codes", con=con)
    icao_list = airport_codes["airport_icao"].tolist()
    flights = tomorrows_flight_arrivals(icao_list)
    flights = flights[
        [
            "flight_number",
            "departure_icao",
            "arrival_icao",
            "arrival_time_local",
            "departure_city",
        ]
    ]
    flights.to_sql("flights", if_exists="append", con=con, index=False)